# (D)DoS operations related to Russia and Ukraine 2022

The goal of this script is to automaticaly analyse (D)DoS attacks that are happening while Russia invates Ukraine in 2022. 
- https://stop-russian-desinformation.near.page/

In [3]:
import pandas as pd

<h1 align='center'>================================================<br>================================================<br>
Analysing: https://stop-russian-desinformation.near.page/</h1>

# Todo:
- About
- DNS resolve (domain to IP)
- IP to ASN/location (IP to ASN)
- Whois (Creation)
- Alexa rank
- Fetch page frequently (every 1h?)
- Analyse the index.html (try to get the code of the attack AND a list of the targets)
- Analyse the .pcap
- Does the IP address of targets change?
- Can we check the status of the pages (up or down)? Evaluate how effective is this #ops

## About
- https://stop-russian-desinformation.near.page/

## Fetching the index.html

In [87]:
import os.path
from os import path

if path.exists("index.html") == False:
    !wget https://stop-russian-desinformation.near.page/
else:
    print("The file index.html was already downloaded!")
 

The file index.html was already downloaded!


## Reading the list of URL's from the index.html

In [88]:
df = pd.read_csv('index.html',sep ='\n')
df = df.rename({'<head>': 'urls'}, axis=1)

In [89]:
df = df[(df.loc[df['urls']=="var targets = ["].index[0]+1) : 
        (df.loc[df['urls']=="]"].index[0])].reset_index(drop=True)

In [90]:
df['urls'] = df['urls'].str.replace("'","").str.replace(",","").str.replace("https://","").str.replace("http://","").str.replace("www.","").str.strip()

<ipython-input-90-b79f56a08a97>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['urls'] = df['urls'].str.replace("'","").str.replace(",","").str.replace("https://","").str.replace("http://","").str.replace("www.","").str.strip()


In [91]:
df['domainname'] = df['urls'].apply(lambda x: x.split('/')[0])

In [94]:
import socket

def domain2ip (df_domains):
    df_output = pd.DataFrame()
    df_domains = df_domains.drop_duplicates()
    for domain in df_domains:
        print(domain)
        try: 
            print(socket.gethostbyname(domain))
        except:
            print("PROBLEM!")
            
    return df_output

In [95]:
domain2ip (df['domainname'])

lenta.ru
178.248.235.122
ria.ru
178.248.233.32
rbc.ru
178.248.234.119
rt.com
185.178.208.5
kremlin.ru
95.173.136.71
en.kremlin.ru
95.173.136.70
smotrim.ru
80.247.32.208
tass.ru
82.202.190.240
tvzvezda.ru
178.248.234.76
vsoloviev.ru
186.2.163.126
1tv.ru
178.248.233.188
vesti.ru
80.247.32.208
online.sberbank.ru
PROBLEM!
sberbank.ru
194.54.14.168
zakupki.gov.ru
95.167.245.92
gosuslugi.ru
109.207.1.118
er.ru
178.248.232.136
rzd.ru
212.164.138.123
rzdlog.ru
185.71.67.190
vgtrk.ru
80.247.32.208
interfax.ru
178.248.233.231
mos.ru
212.11.155.166
government.ru
95.173.136.163
mil.ru
82.202.190.92
nalog.gov.ru
81.176.70.2
customs.gov.ru
89.208.226.45
pfr.gov.ru
195.161.52.80
rkn.gov.ru
185.71.67.220
gazprombank.ru
82.202.190.10
vtb.ru
195.242.83.13
gazprom.ru
217.151.130.37
lukoil.ru
178.248.235.144
magnit.ru
193.19.171.166
nornickel.com
54.72.104.145
surgutneftegas.ru
185.71.67.89
tatneft.ru
185.117.144.192
evraz.com
185.169.155.169
nlmk.com
81.20.194.236
sibur.ru
178.248.232.145
severstal.com
4

In [96]:
socket.gethostbyname('online.sberbank.ru')

'194.54.14.131'

## Enriching IP to ASN

In [13]:
import subprocess
import os.path
import random
import time
impott pandas as pd

def teamcymru_ip2asn (ip):
    command = 'whois -h whois.cymru.com " -v ' + ip + '"'
    output,error = subprocess.Popen(command, universal_newlines=True, shell=True, stdout=PIPE, stderr=PIPE).communicate()
    return output


def teamcymru_ip2asn_bulk(pandaseries_ips):
    iptoasn_response = open('output_ip2asn_teamcymru.txt', 'w')

    #Creating the request file containing a list of IPs
    #For more information, please access: http://www.team-cymru.com/IP-ASN-mapping.html
    iptoasn_request = open('input_ip2asn_teamcymru.txt', 'w')
    iptoasn_request.write('begin\nverbose\n')
    pd.Series(pandaseries_ips.dropna().unique()).to_csv(iptoasn_request,header=False,index=False,sep="\t") 
    iptoasn_request.write('end')
    iptoasn_request.close()

    #Performing the bulk request
    cat = subprocess.Popen(['cat', 'input_ip2asn_teamcymru.txt'], stdout=subprocess.PIPE)
    print(cat)
    netcat = subprocess.Popen(['netcat', 'whois.cymru.com', '43'], stdin=cat.stdout, stdout=iptoasn_response)
    print (netcat)
    time.sleep(3) #for some reason the poll does not work! This was the way to overcome the waiting time.
    #Closing the output file
    iptoasn_response.close()

    df = pd.read_csv('output_ip2asn_teamcymru.txt',\
                     skiprows=1,\
                     delimiter="\s+\|\s",\
                     names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','as_name'])

    ### CAN BE USEFUL: 
    ###df['asn'] = df['asn'].astype(str).apply(lambda x: 'AS'+x.split('.')[0])
    
    !rm input_ip2asn_teamcymru.txt
    !rm output_ip2asn_teamcymru.txt
    
    return df